Spike trains are generated using univariate, self‐exciting Hawkes point process with an exponential kernel, using Ogata’s thinning algorithm. 

In [1]:
import numpy as np
import pandas as pd
import pickle

import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
from cfg_global import project_folder_path
from spike_train_utils import simulate_hawkes_thinning
os.chdir(current_wd)

In [2]:
dataset_folder = project_folder_path + 'results\\synthetic\\dataset\\'

In [3]:
save_data = True

### Main dataset 

1000 spike trains, 3 parameters are fixed 

Fig2 B, C, D

In [ ]:
rate_hz     = 3.5     # target firing rate (Hz)
tau_ms      = 100.0    # desired autocorr time constant (ms)
alpha       = 0.3     # self‐excitation weight (must be <1)
duration_ms = 10*60*1000*1    # simulate for 10 min

# generate
num_trials = 1000
all_spike_trains = []
for trial in range(num_trials):
    spikes = simulate_hawkes_thinning(
        fr_hz_=rate_hz,
        tau_ms_=tau_ms,
        alpha_=alpha,
        duration_ms_=duration_ms,
        seed_=trial  # different seed per trial
    )
    all_spike_trains.append(spikes)

if save_data:
    file_name = 'spike_trains_tau100ms_alpha0_3_fr3_5hz_len600sec_1000.pkl'
    with open(dataset_folder + file_name, 'wb') as f:
        pickle.dump({
            'spike_trains': all_spike_trains,
            'alphas': alpha,
            'fr_values': rate_hz,
            'tau_ms': tau_ms,
            'duration_ms': duration_ms
        }, f, protocol=pickle.HIGHEST_PROTOCOL)

### Parametric dataset 1

10000 spike trains, tau is fixed, fr and alpha are variable

In [ ]:
fr_min, fr_max = 0.01, 10.0  
alpha_min, alpha_max = 0.1, 0.9  

tau_ms      = 100.0    # desired autocorr time constant (ms)
duration_ms = 10*60*1000*1    # simulate for 10 min

# generate
num_trials = 10000

# Random sampling of parameters
global_rng = np.random.default_rng(42)
fr_values = global_rng.uniform(fr_min, fr_max, size=num_trials)
alphas = global_rng.uniform(alpha_min, alpha_max, size=num_trials)

# Generate spike trains
all_spike_trains = []
for trial in range(num_trials):
    rate_hz = fr_values[trial]
    alpha = alphas[trial]
    spikes = simulate_hawkes_thinning(
        fr_hz_=rate_hz,
        tau_ms_=tau_ms,
        alpha_=alpha,
        duration_ms_=duration_ms,
        seed_=trial  # different seed per trial for reproducibility
    )
    all_spike_trains.append(spikes)

if save_data:
    file_name = 'spike_trains_tau100ms_alpha_var_fr_var_len600sec_10000.pkl'
    with open(dataset_folder + file_name, 'wb') as f:
        pickle.dump({
            'spike_trains': all_spike_trains,
            'alphas': alphas,
            'fr_values': fr_values,
            'tau_ms': tau_ms,
            'duration_ms': duration_ms
        }, f, protocol=pickle.HIGHEST_PROTOCOL)

### Parametric dataset 2

100000 spike trains, tau, fr and alpha are variable

In [ ]:
fr_min, fr_max = 0.01, 10.0  
alpha_min, alpha_max = 0.1, 0.9  
tau_ms_min, tau_ms_max = 50, 300

duration_ms = 10*60*1000*1    # simulate for 10 min

# generate
num_trials = 100000

# Random sampling of parameters
global_rng = np.random.default_rng(42)
fr_values = global_rng.uniform(fr_min, fr_max, size=num_trials)
alphas = global_rng.uniform(alpha_min, alpha_max, size=num_trials)
taus = global_rng.uniform(tau_ms_min, tau_ms_max, size=num_trials)

# Generate spike trains
all_spike_trains = []
for trial in range(num_trials):
    rate_hz = fr_values[trial]
    alpha = alphas[trial]
    tau_ms = taus[trial]
    spikes = simulate_hawkes_thinning(
        fr_hz_=rate_hz,
        tau_ms_=tau_ms,
        alpha_=alpha,
        duration_ms_=duration_ms,
        seed_=trial  # different seed per trial for reproducibility
    )
    all_spike_trains.append(spikes)

if save_data:
    file_name = 'spike_trains_3params_var_len600sec_100000.pkl'
    with open(dataset_folder + file_name, 'wb') as f:
        pickle.dump({
            'spike_trains': all_spike_trains,
            'alphas': alphas,
            'fr_values': fr_values,
            'tau_ms': taus,
            'duration_ms': duration_ms
        }, f, protocol=pickle.HIGHEST_PROTOCOL)


### Datasets for Lv tests 

#### Same alpha and tau, diff fr

In [4]:
fr_min, fr_max = 0.01, 10.0  
tau_ms = 100.0   
alpha = 0.3  

duration_ms = 10*60*1000*1    # simulate for 10 min

# generate
num_trials = 1000

# Random sampling of parameters
global_rng = np.random.default_rng(42)
fr_values = global_rng.uniform(fr_min, fr_max, size=num_trials)

# Generate spike trains
all_spike_trains = []
for trial in range(num_trials):
    rate_hz = fr_values[trial]
    spikes = simulate_hawkes_thinning(
        fr_hz_=rate_hz,
        tau_ms_=tau_ms,
        alpha_=alpha,
        duration_ms_=duration_ms,
        seed_=trial  # different seed per trial for reproducibility
    )
    all_spike_trains.append(spikes)

if save_data:
    file_name = 'spike_trains_lv_1param_var_fr_len600sec_1000.pkl'
    with open(dataset_folder + file_name, 'wb') as f:
        pickle.dump({
            'spike_trains': all_spike_trains,
            'alphas': alpha,
            'fr_values': fr_values,
            'tau_ms': tau_ms,
            'duration_ms': duration_ms
        }, f, protocol=pickle.HIGHEST_PROTOCOL)


#### Same fr and tau, different alpha

In [6]:
rate_hz = 3.5 
alpha_min, alpha_max = 0.1, 0.9  
tau_ms = 100.0   

duration_ms = 10*60*1000*1    # simulate for 10 min

# generate
num_trials = 1000

# Random sampling of parameters
global_rng = np.random.default_rng(42)
alphas = global_rng.uniform(alpha_min, alpha_max, size=num_trials)

duration_ms = 10*60*1000*1    # simulate for 10 min

# Generate spike trains
all_spike_trains = []
for trial in range(num_trials):
    alpha = alphas[trial]
    spikes = simulate_hawkes_thinning(
        fr_hz_=rate_hz,
        tau_ms_=tau_ms,
        alpha_=alpha,
        duration_ms_=duration_ms,
        seed_=trial  # different seed per trial for reproducibility
    )
    all_spike_trains.append(spikes)

if save_data:
    file_name = 'spike_trains_lv_1param_var_alpha_len600sec_1000.pkl'
    with open(dataset_folder + file_name, 'wb') as f:
        pickle.dump({
            'spike_trains': all_spike_trains,
            'alphas': alphas,
            'fr_values': rate_hz,
            'tau_ms': tau_ms,
            'duration_ms': duration_ms
        }, f, protocol=pickle.HIGHEST_PROTOCOL)

### Dataset for signal len effect test

In [7]:
rate_hz     = 3.5     
tau_ms      = 100.0   
alpha       = 0.3   

duration_ms_min, duration_ms_max = 2*1000, 10*60*1000 # 2s - 600sec 

num_trials = 10000
global_rng = np.random.default_rng(42)
duration_ms_all = global_rng.integers(duration_ms_min,
                              duration_ms_max,
                              size=num_trials,
                              endpoint=True)

# generate
all_spike_trains = []
for trial in range(num_trials):
    duration_ms = duration_ms_all[trial]
    spikes = simulate_hawkes_thinning(
        fr_hz_=rate_hz,
        tau_ms_=tau_ms,
        alpha_=alpha,
        duration_ms_=duration_ms,
        seed_=trial  # different seed per trial
    )
    all_spike_trains.append(spikes)

if save_data:
    file_name = 'spike_trains_tau100ms_alpha0_3_fr3_5hz_len600sec_var_duration.pkl'
    with open(dataset_folder + file_name, 'wb') as f:
        pickle.dump({
            'spike_trains': all_spike_trains,
            'alphas': alpha,
            'fr_values': rate_hz,
            'tau_ms': tau_ms,
            'duration_ms': duration_ms_all
        }, f, protocol=pickle.HIGHEST_PROTOCOL)